In [5]:
import os
from pymongo import MongoClient
import pandas as pd

# --- Config ---
MONGO_URI = os.getenv("MONGODB_URI", "mongodb://localhost:27017")
DB_NAME = "wowaudit_database"
COLL_NAME = "wowaudit_hourly"
PLAYER_NAME = "Nyph"

# --- Connect ---
client = MongoClient(MONGO_URI)
coll = client[DB_NAME][COLL_NAME]

# --- Query: only what we need, for Nyph ---
pipeline = [
    {"$match": {"metadata.name": PLAYER_NAME}},
    {"$project": {
        "_id": 0,
        "timestamp": 1,
        "season_mythic_dungeons": 1,
        "metadata.expansion": 1,
        "metadata.season": 1
    }},
    {"$sort": {"timestamp": 1}}
]

rows = list(coll.aggregate(pipeline))

# --- Frame it ---
if not rows:
    raise SystemExit(f"No rows found for player {PLAYER_NAME}")

df = pd.DataFrame(rows)

# Flatten metadata fields
df["expansion"] = df["metadata"].apply(lambda m: m.get("expansion") if isinstance(m, dict) else None)
df["season"]    = df["metadata"].apply(lambda m: m.get("season") if isinstance(m, dict) else None)
df = df.drop(columns=["metadata"])

# Ensure timestamp is timezone-aware UTC (Sheets likes ISO strings)
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

# Helpful combined label if you want it in Sheets
df["expansion_season"] = df.apply(
    lambda r: f'{r.get("expansion", "")} {str(r.get("season", ""))}'.strip(),
    axis=1
)

# Optional: per-season "delta" of season_mythic_dungeons
# If season_mythic_dungeons is a running season total (typical), delta shows new dungeons since last sample.
df = df.sort_values(["season", "timestamp"])
df["delta"] = (
    df.groupby("season")["season_mythic_dungeons"]
      .diff()
      .fillna(0)
      .clip(lower=0)
      .astype(int)
)

# Final tidy columns for Google Sheets
out = df[[
    "timestamp",
    "expansion",
    "season",
    "expansion_season",
    "season_mythic_dungeons",
    "delta"
]]

# If you want local CSV to import to Sheets:
out.to_csv("nyph_mplus_by_season.csv", index=False)
print("Wrote nyph_mplus_by_season.csv")

# --- Tips for plotting in Google Sheets ---
# 1) Import the CSV into a sheet.
# 2) Insert → Chart:
#    - For cumulative lines over time: use "timestamp" as X-axis, add a series for "season_mythic_dungeons".
#    - If you want separate lines per season in one chart:
#         * Create a pivot table with Rows = timestamp, Columns = season,
#           Values = MAX of season_mythic_dungeons.
#         * Chart the pivot table as a line chart (one line per season).
#    - For bars of "new dungeons per sample": use the "delta" series instead.


Wrote nyph_mplus_by_season.csv
